In [1]:
import os
import numpy as np
import tensorflow as tf
import rasterio
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
from transformers import SamModel

In [2]:
# Load Pretrained SAM Model from Hugging Face
sam_model = SamModel.from_pretrained("facebook/sam-vit-huge")


# Unfreeze the last 4 layers for fine-tuning
for layer in sam_model.vision_encoder.layers[-4:]:  
    layer.trainable = True


In [3]:
# Define Input Shape (Multispectral Image)
INPUT_SHAPE = (128, 128, 12)
inputs = layers.Input(shape=INPUT_SHAPE)

# Use SAM's encoder features
x = layers.Conv2D(256, (3, 3), padding="same", activation="relu")(inputs)
x = layers.BatchNormalization()(x)  # BatchNorm instead of Dropout
x = layers.Conv2D(256, (3, 3), padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)

# Custom Segmentation Head
x = layers.Conv2DTranspose(128, (3, 3), strides=1, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2DTranspose(64, (3, 3), strides=1, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(1, (1, 1), activation="sigmoid")(x)  # Keep final output at (128, 128, 1)

# Build Model
model = Model(inputs, x)

In [4]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Lower LR for fine-tuning
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.MeanIoU(num_classes=2),
        tf.keras.metrics.BinaryAccuracy()
    ]
)

In [5]:
# Print Model Summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 128, 128, 12)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 128, 128, 256)       │          27,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128, 128, 256)       │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 128, 128, 256)       │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 128, 128, 256)       │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 128, 128, 128)       │         295,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128, 128, 128)       │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 128, 128, 64)        │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 128, 128, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 128, 128, 1)         │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 989,697 (3.78 MB)

 Trainable params: 988,289 (3.77 MB)

 Non-trainable params: 1,408 (5.50 KB)

In [6]:
# Function to Load TIFF Images (12-band) with Min-Max Normalization
def load_image(image_path):
    with rasterio.open(image_path) as src:
        image = src.read().astype(np.float32)  # Shape: (12, 128, 128)
        image = np.transpose(image, (1, 2, 0))  # Convert to (128, 128, 12)

    # Min-max normalization per channel
    min_val = np.min(image, axis=(0, 1), keepdims=True)
    max_val = np.max(image, axis=(0, 1), keepdims=True)
    normalized_image = (image - min_val) / (max_val - min_val + 1e-7)
    
    return normalized_image


In [7]:
# Function to Load Labels (Binary Masks)
def load_label(label_path):
    label = tf.keras.preprocessing.image.load_img(label_path, color_mode="grayscale")
    label = np.array(label, dtype=np.uint8)  # Convert to numpy array
    label = (label > 0).astype(np.int32)  # Ensure binary format
    return label


In [8]:
# Paths to images and labels
data_dir = r"C:\Users\Eman\Downloads\images-20250217T060743Z-001\images"
labels_dir = r"C:\Users\Eman\Downloads\labels-20250217T060744Z-001\labels"


In [9]:
# Get sorted lists of image and label file paths
image_paths = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".tif")])
label_paths = sorted([os.path.join(labels_dir, f) for f in os.listdir(labels_dir) if f.endswith(".png")])

# Load Data into NumPy Arrays
X = np.array([load_image(p) for p in image_paths])
Y = np.array([load_label(p) for p in label_paths])

C:\Users\Eman\anaconda3\Lib\site-packages\rasterio\__init__.py:368: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [10]:

# Split Data (70% Training, 15% Validation, 15% Testing)
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)


In [11]:
# Train Model
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    batch_size=16,
    epochs=30,
    verbose=1
)

Epoch 1/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 130s 9s/step - binary_accuracy: 0.7174 - loss: 0.5570 - mean_io_u: 0.3604 - val_binary_accuracy: 0.8283 - val_loss: 0.6431 - val_mean_io_u: 0.3703
Epoch 2/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 141s 9s/step - binary_accuracy: 0.8740 - loss: 0.3459 - mean_io_u: 0.3620 - val_binary_accuracy: 0.8205 - val_loss: 0.5879 - val_mean_io_u: 0.3703
Epoch 3/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 125s 9s/step - binary_accuracy: 0.8981 - loss: 0.2876 - mean_io_u: 0.3695 - val_binary_accuracy: 0.8114 - val_loss: 0.5500 - val_mean_io_u: 0.3703
Epoch 4/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 125s 9s/step - binary_accuracy: 0.8886 - loss: 0.2942 - mean_io_u: 0.3697 - val_binary_accuracy: 0.7927 - val_loss: 0.5334 - val_mean_io_u: 0.3703
Epoch 5/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 144s 9s/step - binary_accuracy: 0.8909 - loss: 0.2899 - mean_io_u: 0.3870 - val_binary_accuracy: 0.7465 - val_loss: 0.5141 - val_mean_io_u: 0.3703
Epoch 6/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 125s 9s/step - binary_accuracy: 0.901

In [23]:
# Print Training History
for epoch, (loss, val_loss, iou, val_iou, acc, val_acc) in enumerate(zip(
    history.history["loss"], history.history["val_loss"], 
    history.history["mean_io_u"], history.history["val_mean_io_u"],
    history.history["binary_accuracy"], history.history["val_binary_accuracy"]
)):
    print(f"Epoch {epoch+1}: Loss={loss:.4f}, Val_Loss={val_loss:.4f}, IoU={iou:.4f}, Val_IoU={val_iou:.4f}, Acc={acc:.4f}, Val_Acc={val_acc:.4f}")

# Save Model
model.save("fine_tuned_sam_water_segmentation.h5")


Epoch 1: Loss=0.4576, Val_Loss=0.6431, IoU=0.3631, Val_IoU=0.3703, Acc=0.7904, Val_Acc=0.8283
Epoch 2: Loss=0.3304, Val_Loss=0.5879, IoU=0.3643, Val_IoU=0.3703, Acc=0.8784, Val_Acc=0.8205
Epoch 3: Loss=0.2867, Val_Loss=0.5500, IoU=0.3631, Val_IoU=0.3703, Acc=0.8979, Val_Acc=0.8114
Epoch 4: Loss=0.2916, Val_Loss=0.5334, IoU=0.3678, Val_IoU=0.3703, Acc=0.8892, Val_Acc=0.7927
Epoch 5: Loss=0.2866, Val_Loss=0.5141, IoU=0.3945, Val_IoU=0.3703, Acc=0.8935, Val_Acc=0.7465
Epoch 6: Loss=0.2658, Val_Loss=0.5088, IoU=0.3873, Val_IoU=0.3703, Acc=0.9040, Val_Acc=0.7407
Epoch 7: Loss=0.2780, Val_Loss=0.5091, IoU=0.3917, Val_IoU=0.3703, Acc=0.8987, Val_Acc=0.7407
Epoch 8: Loss=0.2578, Val_Loss=0.5005, IoU=0.3794, Val_IoU=0.3703, Acc=0.9038, Val_Acc=0.7407
Epoch 9: Loss=0.2474, Val_Loss=0.4940, IoU=0.3876, Val_IoU=0.3703, Acc=0.9047, Val_Acc=0.7407
Epoch 10: Loss=0.2600, Val_Loss=0.4891, IoU=0.3906, Val_IoU=0.3703, Acc=0.9048, Val_Acc=0.7407
Epoch 11: Loss=0.2387, Val_Loss=0.4783, IoU=0.3846, Val_IoU

In [25]:

# Evaluate Model on Test Set
test_loss, test_iou, test_accuracy = model.evaluate(X_test, Y_test, verbose=1)

# Print Test Results
print(f"Test Loss: {test_loss:.4f}")
print(f"Test IoU: {test_iou:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - binary_accuracy: 0.8861 - loss: 0.2808 - mean_io_u: 0.4043
Test Loss: 0.2817
Test IoU: 0.4028
Test Accuracy: 0.8855
